In [1]:
from glob import glob
import numpy as np
import os
import pickle

In [2]:
neighbor_threshold = 40

In [3]:
pkl_list = os.listdir("./train")

In [16]:
X, y, max_vecs, min_vecs, ID = None, None, None, None, None
for fname in pkl_list:
    with open("./train/" + fname, "rb") as f:
        data = pickle.load(f)
        car_mask = np.where(data["car_mask"] == 1)[0]
        pred_target = np.where(data["track_id"] == data["agent_id"])[0][0]
        if pred_target not in car_mask:
            continue
        input_feature = np.concatenate([data["p_in"][car_mask], data["v_in"][car_mask]], axis=2)
        target_feature = np.concatenate([data["p_in"][pred_target], data["v_in"][pred_target]], axis=1)
        
        # feature 1 normalized location
        max_vec = np.max(input_feature, axis=(0, 1))
        min_vec = np.min(input_feature, axis=(0, 1))
        normalized_location = (target_feature - min_vec) / (max_vec - min_vec)
        
        # feature 2 distance with respect to other cars (top 2)
        car_distance_per_time = np.linalg.norm((input_feature - target_feature)[:, :, :2], axis=2).T
        closest_distances = np.sort(car_distance_per_time, axis=1)[:, 1:2] # 1 car
        
        # feature 3 number of neighbors within 
        num_neighbors = np.sum(car_distance_per_time > neighbor_threshold, axis=1).reshape(-1, 1)
        
        input_X = np.hstack([normalized_location, closest_distances, num_neighbors])
        target_y = data["p_out"][pred_target]
        target_y = (target_y - min_vec[:2]) / ((max_vec - min_vec)[:2])
        
        input_X, target_y = input_X.reshape(-1), target_y.reshape(-1)
        if X is None:
            X = input_X
            y = target_y
            max_vecs = max_vec[:2]
            min_vecs = min_vec[:2]
            ID = data["scene_idx"]
        else:
            X = np.vstack([X, input_X])
            y = np.vstack([y, target_y])
            max_vecs = np.vstack([max_vecs, max_vec[:2]])
            min_vecs = np.vstack([min_vecs, min_vec[:2]])
            ID = np.append(ID, data["scene_idx"])
        if len(X) % 1000 == 0:
            print(len(X))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [18]:
np.savez("processed_training.npz", X=X, y=y, max_vecs=max_vecs, min_vecs=min_vecs, ID=ID)

In [ ]:
pkl_list = os.listdir("./val_in")

In [ ]:
'''
X, y, ID = None, None, None
for fname in pkl_list:
    with open("./val_in/" + fname, "rb") as f:
        data = pickle.load(f)
        scene_idx = data["scene_idx"]
        car_mask = np.where(data["car_mask"] == 1)[0]
        pred_target = np.where(data["track_id"] == data["agent_id"])[0][0]
        if pred_target not in car_mask:
            continue
        input_feature = np.concatenate([data["p_in"][car_mask], data["v_in"][car_mask]], axis=2)
        target_feature = np.concatenate([data["p_in"][pred_target], data["v_in"][pred_target]], axis=1)
        target_feature = (target_feature - np.mean(input_feature, axis=(0, 1))) / np.std(input_feature, axis=(0, 1))
        target_feature = target_feature.reshape(-1)
        if X is None:
            X = target_feature
            ID = data["scene_idx"]
        else:
            X = np.vstack([X, target_feature])
            ID = np.append(ID, data["scene_idx"])
        if len(X) % 1000 == 0:
            print(len(X))
'''

In [ ]:
'''
np.savez("processed_testing.npz", X=X, y=y, ID=ID)
'''